In [ ]:
"""

    List How many queues in SQS service 
    Create a queue
    List How many queues in SQS service 
    List messages in queue
    Put a message into queue
    List messages in queue
    Get a message from queue
    List messages in queue (the message invisible)
    wait 10 seconds
    List messages in queue (the message visible)
    Get a message from queue and delete message from queue
    wait 10 second
    List messages in queue (the message visible)


"""

In [ ]:
# !pip install boto3 awscli

In [ ]:
import boto3

# Get the service resource
sqs_client = boto3.client('sqs', endpoint_url='http://localstack-main:4566')


In [ ]:
# List How many queues in SQS service 

print(sqs_client.list_queues())
print(sqs_client.list_queues().get('QueueUrls'))


In [ ]:
# Create a queue

response = sqs_client.create_queue(
    QueueName='whatever_you_like',
)
response


# # Deletes the queue specified by the QueueUrl , 
# #     regardless of the queue's contents. 

# response = sqs_client.delete_queue(
#     QueueUrl='http://localhost:4566/queue/whatever_you_like'
# )
# response

In [ ]:
# List How many queues in SQS service 

print(sqs_client.list_queues())
print(sqs_client.list_queues().get('QueueUrls'))


In [ ]:
# List messages in queue
response = sqs_client.receive_message(
    QueueUrl='http://localhost:4566/queue/whatever_you_like',
    MaxNumberOfMessages=10, # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=3,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)。
)

response

In [ ]:
# Put a message into queue

# MessageBody (string) [REQUIRED]
# The message to send. The maximum string size is 256 KB.

response = sqs_client.send_message(QueueUrl='http://localhost:4566/queue/whatever_you_like',
                               MessageBody='this is a message, please get it..')
response

In [ ]:
# List messages in queue

response = sqs_client.receive_message(
    QueueUrl='http://localhost:4566/queue/whatever_you_like',
    MaxNumberOfMessages=10, # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=0,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)。
)

response

In [ ]:
# Get a message from queue
response = sqs_client.receive_message(
    QueueUrl='http://localhost:4566/queue/whatever_you_like',
    MaxNumberOfMessages=1, # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=5,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)。
)

response.get('Messages')

In [ ]:
# List messages in queue (the message invisible)
# wait 10 seconds
# List messages in queue (the message visible)

# https://docs.aws.amazon.com/zh_tw/AWSSimpleQueueService/latest/SQSDeveloperGuide/sqs-delay-queues.html
# 可見性逾時 VisibilityTimeout: 
#     接收消息時設定，訊息的預設可見性逾時為 30 秒。下限為 0 秒。上限為 12 小時。

import time

response = sqs_client.receive_message(
    QueueUrl='http://localhost:4566/queue/whatever_you_like',
    MaxNumberOfMessages=10, # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=0,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)。
    VisibilityTimeout=10,
)
print("有多少訊息: ", len(response.get('Messages')) if response.get('Messages') else 0)
print(response.get('Messages'))

# time.sleep(10)

# response = sqs_client.receive_message(
#     QueueUrl='http://localhost:4566/queue/whatever_you_like',
#     MaxNumberOfMessages=10, # Retrieves one or more messages (up to 10)
#     WaitTimeSeconds=5,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)。
#     VisibilityTimeout=10,
# )
# print(response)


In [ ]:
# Get a message from queue and delete message from queue
# wait 10 second
# List messages in queue (the message visible)

# 取得一筆訊息
response = sqs_client.receive_message(
    QueueUrl='http://localhost:4566/queue/whatever_you_like',
    MaxNumberOfMessages=1, # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=0,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)。
    VisibilityTimeout=0,
)
print(response.get('Messages'))
print(response.get('Messages')[0].get('ReceiptHandle'))

# 使用ReceiptHandle 刪除此訊息，每次接收訊息 ReceiptHandle 都會不一樣
response = sqs_client.delete_message(
    QueueUrl='http://localhost:4566/queue/whatever_you_like',
    ReceiptHandle=response.get('Messages')[0].get('ReceiptHandle')
)
print('刪除訊息: ', response)

# 再次收取訊息
response = sqs_client.receive_message(
    QueueUrl='http://localhost:4566/queue/whatever_you_like',
    MaxNumberOfMessages=10, # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=0,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)。
    VisibilityTimeout=0,  
)
print(response.get('Messages'))